# Deep Network
# This training is super super slow, it can take up hours!!!

Originally by the Tensorflow authors, modified by Donald Li for Fall 2017

If you find this ipython notebook is unclear or contains bugs, please contact Donald at sli97@jhu.edu  
This homework requires [tensorflow](https://www.tensorflow.org/install/), please make sure it is correctly installed before running the codes.

The data and code can be downloaded here: [tgz](https://github.com/shipui2005/ProbHW5/blob/master/HW5.tar.gz) or [zip](https://github.com/shipui2005/ProbHW5/blob/master/HW5.zip)
convolutional deep neural network to classify hand-written digits. This is a The training project will require some time to run.

# Initialization and Preparing DATA


In [22]:
#Start a new session for tensorflow
import tensorflow as tf
sess = tf.InteractiveSession()

#Download Mnist dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

#Initialize weight
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

#Initialize place holder
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Here, we define what a convolutional layer and pooling layers. The following is the structure of the current convolutional network.
<img width="300" src='mnist_deep.png'>

In [15]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

def avg_pool_2x2(x):
  return tf.nn.avg_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


We have to specify how many parameters there should be, and for the first convolution and pooling layer, what kind of how many units are there, and how they are being connected. Here, h is the hidden layer.

In [16]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1, 28, 28, 1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)
#h_pool1 = avg_pool_2x2(h_conv1)

Of course, we can do similar thing to make the second layer of the network, it is important to notice that the number of input and output units are differnet in different layers, so when you want to decrease or increase the number of layer, you have to be careful about that. The output image size is 7*7 here.

In [17]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)
#h_pool2 = avg_pool_2x2(h_conv2)

We add a fully-connected layer to allow processing on the entire image, here we have used a ReLu function. Then do the drop out process to prevent overfitting, finally there is a read out layer to get the results. 

In [18]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

Here, we train and test the model. Instead of using least square, we are using cross entropy to be the term being oprtimized. We do not use a steepest gradient descent here, but using a ADAM alogrithm to do the optimization, in every 100 iterations, results will be printed out. For each batch of training, we feed in 50 images. This training is super super slow, it can take up hours!!!

In [19]:
# Defining cross-entropy here
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#Do the training
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
      train_accuracy = accuracy.eval(feed_dict={
          x: batch[0], y_: batch[1], keep_prob: 1.0})
      print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

#Test the accuracy
  print('test accuracy %g' % accuracy.eval(feed_dict={
      x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.14
step 100, training accuracy 0.9


KeyboardInterrupt: 

# Softmax Regression Model

First, we have to initialize place holders and variables. 
It will be better to run the initialization and data preparation cell again before running the following!

In [24]:
#Initialize weights and bias for the regression model
#Start a new session for tensorflow
import tensorflow as tf
sess = tf.InteractiveSession()

#Download Mnist dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

#Initialize place holder
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])


W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

sess.run(tf.global_variables_initializer())

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


The class is predicted using a softmax model, and the loss function uses a cross entropy in this case.

In [25]:
#Class prediction
y = tf.matmul(x,W) + b

#Loss function
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

The following cell do the training for the regression model, you can try to manipulate the number of training done to see what's happen.

In [26]:
#Do the gradient decent training 
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
for _ in range(10): #You may want to try 10,100,1000,10000
  batch = mnist.train.next_batch(100)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

After training, calculate the prediction and see how off it is to the true value.

In [27]:

#Check the prediction
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

#Calculate and print the accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy is", accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

('Accuracy is', 0.76029998)


# HW

1. Try both average pooling and max-pooling for the convolutional neural network, does the results agree on what you have discussed in Question 2-2? What is the difference between them? (5 points)
2. Modify the code to make it into a one-layer network (it is very simple), but you just have to be careful about the dimension of each output layer to make sure they are correct, do you find a accuracy difference between one-layer convolutional network and a two-layer convolutional network? Report the results and explain (5 points)
3. Run the simple softmax regression model and see how is the results different from the convolutional neural network? You should change the number of training and see how the accuaracy changes in the regression model. (5 points)